# Merge And Graph
This script loads all the measures tables generated, calculates the bootstraps and plots it


### Imports + Global Setup

In [4]:
from __future__ import print_function, division

import sys 

import numpy as np
import pandas as pd
import re
import os
import warnings
import math

from operator import itemgetter

# import pyedf
# import conf

pd.options.display.mpl_style = 'default'

plt.rcParams['figure.figsize'] = (16.0, 10.0)
plt.set_cmap('gnuplot2')



C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


## Global Constants

In [5]:
# path = './DataDummy'
#path = r'G:\Miguel\Research\Research-Current\CMOS\Git Experiment Only\Logs (For test analysis)'
#path = r'G:\Miguel\Research\Research-Current\CMOS\pilot data'
path = ".\processedData"
#participant_ids = [ 'None', 'Johanas' ]
participant_ids = [  '15FM24',
                     '21MDC20',
                     '3MW2421',
                     '22F',
                     '14JG23',
                     '1MQ2201',
                     '2ML3603',
                     '8FC02',
                     '5MS3205',
                     '9FH04',
                     '1MW2207',
                     '12FRH06',
                     '6MDH09',
                     '13MM08',
                     '12JR11',
                     '10F',
                     '17MRW13',
                     '12F',
                     'MPS15',
                     '2FQ2014',
                     '5MS17',
                     '2FS2116',
                     '18MDM19',
                     '18F']
condition_ids = ['e', 'c', 's', 'n']
#condition_ids = ['e', 'c']
mediaObjectsPerDocument = 20 # this comes from how the documents were set up.
mediaObjectsScotland = 4 # this comes from how the documents were set up.

## Loading Calculated Data 

In [6]:
pathEyeResults = path+'\\'+"fixationAnalysisResults_pandas.pddf"
pathQuestionnaireResults = path+'\\'+'TLX_and_Questions_AnalysisResults_pandas.pddf'
pathInteractionResults = path+'\\'+'interactionAnalysisResults_pandas.pddf'

frameEye = pd.read_pickle(pathEyeResults)
frameQuestionnaire = pd.read_pickle(pathQuestionnaireResults)
frameInteraction = pd.read_pickle(pathInteractionResults)

#frameEyeFiltered = frameEye

allData = pd.concat([frameEye,frameQuestionnaire,frameInteraction], axis = 1)
#allData = frameEye.append([frameQuestionnaire,frameInteraction])







In [13]:
# Change order of columns before saving
cols = ['userId','training','country','orderNumber','experimentNumber','type','M1','M2','M2country','M3','M3m','M3o','M3t',
    'M4','M4m','M4o','M4t','M4perSec','M5','M5m','M5o','M5t',
    'M6','M6m','M6o','M6t','M7','M8','M9','M10','M11','M12','M12m','M12o','M12t',
    'M13m','M13o','M13t','M14','M15','M15m','M15o','M15t','M16','M16m','M16o','M16t','M17',
    'M18','M19','M20','M21','M22','M23','M24','M25','M26','M29','M30','M30country','V10t']

df = allData[cols]

# print(df)

df.to_pickle(r'.\processedData\all_trials_processed_data_pandas.pddf')
df.to_csv(r'.\processedData\all_trials_processed_data.csv')
df.to_excel(r'.\processedData\all_trials_processed_data.xls')

      userId   userId   userId training training training  country  country  \
0     15FM24   15FM24   15FM24    False    False    False        0        0   
1     15FM24   15FM24   15FM24    False    False    False        3        3   
2     15FM24   15FM24   15FM24    False    False    False        0        0   
3     15FM24   15FM24   15FM24    False    False    False        4        4   
4     15FM24   15FM24   15FM24    False    False    False        0        0   
5     15FM24   15FM24   15FM24    False    False    False        1        1   
6     15FM24   15FM24   15FM24    False    False    False        0        0   
7     15FM24   15FM24   15FM24    False    False    False        2        2   
8    21MDC20  21MDC20  21MDC20    False    False    False        0        0   
9    21MDC20  21MDC20  21MDC20    False    False    False        2        2   
10   21MDC20  21MDC20  21MDC20    False    False    False        0        0   
11   21MDC20  21MDC20  21MDC20    False    False    

In [11]:
pd.set_option('display.max_columns', None)
frameEye[:5]
allData.shape

(190, 73)

In [ ]:
frameQuestionnaire[:5]

In [ ]:
frameInteraction[:5]